In [1]:
# """
# View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
# My Youtube Channel: https://www.youtube.com/user/MorvanZhou
# More about Reinforcement learning: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/
# Dependencies:
# torch: 0.4
# gym: 0.8.1
# numpy
# """

In [2]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import matplotlib.pyplot as plt

In [3]:
seed = 161
# random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
# os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000

In [5]:
env = gym.make('CartPole-v0')
env._max_episode_steps = 10
env = env.unwrapped

In [6]:
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

In [7]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [8]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
#         print("Neural net")
#         print(self.eval_net)

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action
    
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        actions_value = self.eval_net.forward(x)
        action = torch.max(actions_value, 1)[1].data.numpy()
        action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [9]:
dqn = DQN()
NO_OF_EPISODES = 500

In [10]:
print('\nCollecting experience...')
ep_r_rec = []
for i_episode in range(NO_OF_EPISODES):
    s = env.reset()
    ep_r = 0
    while True:
#         env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
#             if done:
#                 print('Ep: ', i_episode,
#                       '| Ep_r: ', round(ep_r, 2))

        if done:
            ep_r_rec = np.append(ep_r_rec,ep_r)
            print('Ep: ', i_episode,
                  '| Ep_r: ', round(ep_r, 2))
            
            break
        s = s_


Ep:  0 | Ep_r:  2.54
Ep:  1 | Ep_r:  1.73
Ep:  2 | Ep_r:  2.33
Ep:  3 | Ep_r:  2.75
Ep:  4 | Ep_r:  2.31
Ep:  5 | Ep_r:  2.52
Ep:  6 | Ep_r:  3.9
Ep:  7 | Ep_r:  2.3
Ep:  8 | Ep_r:  2.49
Ep:  9 | Ep_r:  3.1
Ep:  10 | Ep_r:  1.57
Ep:  11 | Ep_r:  1.43
Ep:  12 | Ep_r:  3.14
Ep:  13 | Ep_r:  1.87
Ep:  14 | Ep_r:  1.63
Ep:  15 | Ep_r:  1.11
Ep:  16 | Ep_r:  2.47
Ep:  17 | Ep_r:  2.18
Ep:  18 | Ep_r:  1.87
Ep:  19 | Ep_r:  3.13
Ep:  20 | Ep_r:  2.47
Ep:  21 | Ep_r:  2.87
Ep:  22 | Ep_r:  2.86
Ep:  23 | Ep_r:  2.85
Ep:  24 | Ep_r:  2.79
Ep:  25 | Ep_r:  1.89
Ep:  26 | Ep_r:  2.5
Ep:  27 | Ep_r:  2.6
Ep:  28 | Ep_r:  2.39
Ep:  29 | Ep_r:  2.84
Ep:  30 | Ep_r:  2.74
Ep:  31 | Ep_r:  4.39
Ep:  32 | Ep_r:  2.79
Ep:  33 | Ep_r:  2.98
Ep:  34 | Ep_r:  0.94
Ep:  35 | Ep_r:  3.07
Ep:  36 | Ep_r:  5.13
Ep:  37 | Ep_r:  1.84
Ep:  38 | Ep_r:  1.44
Ep:  39 | Ep_r:  2.67
Ep:  40 | Ep_r:  4.12
Ep:  41 | Ep_r:  1.06
Ep:  42 | Ep_r:  1.14
Ep:  43 | Ep_r:  2.34
Ep:  44 | Ep_r:  3.05
Ep:  45 | Ep_r:  2.4
Ep:

KeyboardInterrupt: 

In [ ]:
plt.plot(ep_r_rec);

In [ ]:
env.close()

In [ ]:
# s = env.reset()
# while True:
#     env.render()
#     a = dqn.choose_greedy_action(s)
#     s_, r, done, info = env.step(a)
#     if done:
#         break
#     s = s_

In [ ]:
# env.close()